In [1]:
!pip install optuna


In [2]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords, wordnet
from wordcloud import WordCloud, STOPWORDS
from nltk.stem import WordNetLemmatizer
import os
import string
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
from sklearn.decomposition import LatentDirichletAllocation
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
# import plotly.express as px
from sklearn.manifold import TSNE
import itertools
import collections
from collections import Counter
from gensim import corpora, models
from gensim.models import Word2Vec, word2vec, LdaModel
import optuna
from hyperopt import hp, tpe, fmin
import warnings
warnings.filterwarnings("ignore")
import gensim
from gensim.utils import simple_preprocess
from datetime import datetime
# import spacy
from nltk import pos_tag
import pickle


In [3]:
nltk.download('averaged_perceptron_tagger')
lemmatizer = WordNetLemmatizer()
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
# nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
df = pd.read_csv('drive/MyDrive/5.TopicModeling/Data/2.TubbsFire_wildfire_words.csv', encoding='latin-1')
df['tweet_created_at'] = pd.to_datetime(df["tweet_created_at"])
df['tweet_created_at'] = df['tweet_created_at'].dt.strftime('%Y-%m-%d')
df #splitting does not apply to this case

,tweet_id,tweet_text,tweet_created_at,tweet_source,tweet_lang,tweet_possibly_sensitive,tweet_retweet_count,tweet_reply_count,tweet_like_count,tweet_quote_count,...,user_friends_count,user_tweet_count,place_id,place_name,place_full_name,place_country,place_country_code,place_type,clean_text,tokens
0,9.041318e+17,"#WILDFIRE brings smoke, ash, unhealthful air q...",2017-09-02,NaN,en,True,0.0,0.0,0.0,0.0,...,51.0,101051.0,NaN,NaN,NaN,NaN,NaN,NaN,wildfire brings smoke ash unhealthful air qual...,"['wildfire', 'brings', 'smoke', 'ash', 'unheal..."
1,9.041317e+17,"#WILDFIRE brings smoke, ash, unhealthful air q...",2017-09-02,NaN,en,True,0.0,0.0,0.0,0.0,...,9203.0,113971.0,NaN,NaN,NaN,NaN,NaN,NaN,wildfire brings smoke ash unhealthful air qual...,"['wildfire', 'brings', 'smoke', 'ash', 'unheal..."
2,9.041309e+17,California on fire and Texas under water smh,2017-09-02,NaN,en,True,0.0,0.0,13.0,0.0,...,573.0,3884.0,NaN,NaN,NaN,NaN,NaN,NaN,california fire texas water smh,"['california', 'texas', 'water', 'smh']"
3,9.041304e+17,MVC: 6000 block of Tom Higgins Lane. Lake Shan...,2017-09-02,NaN,en,True,0.0,0.0,0.0,0.0,...,1016.0,86875.0,NaN,NaN,NaN,NaN,NaN,NaN,mvc 6000 block tom higgins lane lake shannon b...,"['mvc', 'block', 'tom', 'higgins', 'lane', 'la..."
4,9.041302e+17,"Wildlife brings smoke, ash, unhealthful air qu...",2017-09-02,NaN,en,True,0.0,0.0,0.0,0.0,...,595.0,48048.0,NaN,NaN,NaN,NaN,NaN,NaN,wildlife brings smoke ash unhealthful air qual...,"['wildlife', 'brings', 'smoke', 'ash', 'unheal..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
301471,9.415234e+17,"Pasadena, California â¢ \nJet Propulsion Labo...",2017-12-15,NaN,en,False,3.0,0.0,9.0,0.0,...,590.0,2848.0,0775fcd6eb188d7c,Mid-Wilshire,"Mid-Wilshire, Los Angeles",Verenigde Staten,US,neighborhood,pasadena california jet propulsion laboratory ...,"['pasadena', 'california', 'jet', 'propulsion'..."
301472,9.419101e+17,So sad.........:(\nPlease help the family of t...,2017-12-16,NaN,en,True,0.0,0.0,0.0,0.0,...,600.0,11015.0,NaN,NaN,NaN,NaN,NaN,NaN,sad please help family california firefighter ...,"['sad', 'help', 'family', 'california', 'firef..."
301473,9.424684e+17,My heart goes out to all those affected by the...,2017-12-17,NaN,en,True,0.0,0.0,1.0,0.0,...,417.0,34379.0,NaN,NaN,NaN,NaN,NaN,NaN,heart goes affected california wildfires sad,"['heart', 'goes', 'affected', 'california', 'w..."
301474,9.458634e+17,@realDonaldTrump Thank you Mr Trump for your i...,2017-12-27,NaN,en,True,0.0,0.0,0.0,0.0,...,25.0,158.0,NaN,NaN,NaN,NaN,NaN,NaN,thank mr trump incredible leadership divine sk...,"['thank', 'mr', 'trump', 'incredible', 'leader..."


# Preprocess

In [6]:
def preprocess(text):
  # Cleaning everything except alphabetical and numerical characters
  # text = ' '.join(text)
  text = re.sub(r'@\S+', ' ', text) #remove @
  text = re.sub(r'#\S+', ' ', text) #remove hashtag
  text = re.sub(r'http\S+', ' ', text) #remove hyperlink
  text = re.sub("[^a-zA-Z0-9]"," ",text) #keep only letters and numbers
  tokens = nltk.word_tokenize(text.lower()) #tokenize
  tokens = [i for i in tokens if len(i) > 2]
  filtered_tokens = [token for token in tokens if token not in stop_words]  # Removing stopwords
  tagged_tokens = pos_tag(filtered_tokens)
  return tagged_tokens

def lemmatize_text(tagged_tokens):
    lemmatized_tokens = []
    for token, tag in tagged_tokens:
        if tag.startswith('N'):  # Noun
            lemmatized_tokens.append(lemmatizer.lemmatize(token, pos='n'))
        elif tag.startswith('V'):  # Verb
            lemmatized_tokens.append(lemmatizer.lemmatize(token, pos='v'))
        elif tag.startswith('R'):  # Adverb
            lemmatized_tokens.append(lemmatizer.lemmatize(token, pos='r'))
        elif tag.startswith('J'):  # Adjective
            lemmatized_tokens.append(lemmatizer.lemmatize(token, pos='a'))
        else:
            lemmatized_tokens.append(None)
    return lemmatized_tokens
CleanText = [preprocess(text) for text in df['tweet_text']]
df['token'] = CleanText
lemmatized_tokens = [lemmatize_text(tokens) for tokens in df['token']]
filtered_lemmatized_tokens = [[token for token in tokens if token is not None] for tokens in lemmatized_tokens]
df['lemm_text'] = filtered_lemmatized_tokens

In [8]:
# df.to_csv( "drive/MyDrive/5.TopicModeling/Data/final.csv", encoding='latin-1')